first import the library


In [191]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

load the dataset

In [192]:
df=pd.read_csv("ev_charging_patterns.csv")
#df.head()
df.isnull().sum()
# 5. Drop rows with nulls (if necessary)
df = df.dropna()
df.isnull().sum()
df.shape

(1131, 20)

In [193]:
df.describe()

,Battery Capacity (kWh),Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years)
count,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000,1131.000000
mean,74.427818,42.915668,2.303177,26.014166,22.488351,49.230036,75.012917,153.663101,15.305780,3.604227
std,20.828350,22.201286,1.065878,14.010292,10.792504,24.170435,16.920463,85.549751,14.751266,2.324090
min,1.536540,0.045772,0.095314,1.472549,0.307085,2.325959,7.604224,1.899538,-10.724770,0.000000
25%,62.000000,24.248936,1.425281,13.949809,13.133925,27.661992,62.264460,80.954993,3.009498,2.000000
50%,75.000000,42.865611,2.312675,25.838488,21.828088,48.947886,75.100944,152.257515,14.641853,4.000000
75%,85.000000,61.544055,3.145998,37.508677,31.675804,69.783816,88.245070,225.469628,27.824244,6.000000
max,193.003074,152.238758,7.635145,97.342255,69.407743,125.087227,177.708666,398.364775,73.169588,11.688592


In [194]:
df.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


data split

In [207]:


#sns.pairplot(df, kind='scatter',plot_kws={'alpha':0.5})
#plt.show()



laben=LabelEncoder()
df['Vehicle Model']=laben.fit_transform(df['Vehicle Model'])
df['User Type']=laben.fit_transform(df['User Type'])
df['Charger Type']=laben.fit_transform(df['Charger Type'])
df['Time of Day']=laben.fit_transform(df['Time of Day'])
df['Day of Week']=laben.fit_transform(df['Day of Week'])


drop=["User ID","Charging Station ID","Charging Station Location","Temperature (°C)","Vehicle Age (years)","Distance Driven (since last charge) (km)","Charging Start Time","Charging End Time"]
x=df.drop(columns=drop)
y=df["Charging Cost (USD)"]
y

0       13.087717
1       21.128448
2       35.667270
3       13.036239
4       10.161471
          ...    
1315    22.081164
1316     5.067806
1317    37.255002
1318    39.046146
1319    10.863674
Name: Charging Cost (USD), Length: 1131, dtype: float64

training the model

In [196]:
X=np.asarray(x)
Y=np.asarray(y)
model=LinearRegression()
model.fit(x,y)
model.intercept_, model.coef_
y_pred = model.predict(X)
residuals = y - y_pred

# Step 2: Compute gradients
gradients = 2 * residuals

# Step 3: Define outlier threshold
threshold = np.percentile(np.abs(gradients), 95)  # Top 5% gradients as outliers

# Step 4: Remove outliers
outliers = np.abs(gradients) > threshold
X_filtered = X[~outliers]
y_filtered = y[~outliers]


#Step 5: Refit the model
model_filtered = LinearRegression()
model_filtered.fit(X_filtered, y_filtered)


c:\Users\jk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


LinearRegression()

In [197]:
x_train, y_train, x_test, y_test=train_test_split(X_filtered,y_filtered, test_size=0.2,random_state=2)
model_filtered = LinearRegression()
model_filtered.fit(X_filtered, y_filtered)
model_filtered.intercept_, model_filtered.coef_


(np.float64(-4.973799150320701e-14),
 array([ 1.88747637e-15,  1.11022302e-16,  9.71445147e-17, -7.63278329e-16,
         2.39391840e-16,  1.00000000e+00,  2.16515174e-16, -5.16947596e-16,
         1.38777878e-16, -9.36750677e-17, -1.09287579e-16,  2.41993925e-16]))

In [198]:
train_data_prediction=model.predict(X_filtered)
# Calculate various accuracy metrics
r2 = r2_score(y_filtered, train_data_prediction)
mse = mean_squared_error(y_filtered, train_data_prediction)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_filtered, train_data_prediction)

# Print the results
print(f"R-squared Score: {r2:.4f}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")

R-squared Score: 1.0000
Mean Squared Error: 0.0000
Root Mean Squared Error: 0.0000
Mean Absolute Error: 0.0000


c:\Users\jk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


 code deploy on streamlit  

In [202]:
import streamlit as st
import numpy as np

def main():
    st.title('Electric Vehicle Charging Cost Prediction')
    
    # Create a form
    with st.form("prediction_form"):
        st.subheader("Input Parameters")
        
        # 1. Vehicle Model
        vehicle_model = st.selectbox(
            "Vehicle Model",
            options=[0, 1, 2, 3, 4],
            format_func=lambda x: {
                0: "BMW i3",
                1: "Chery Bolt",
                2: "Hyundai Kona",
                3: "Nissan Leaf",
                4: "Tesla Model 3"
            }[x],
            help="Select your vehicle model"
        )
        
        # Create two columns for better layout
        col1, col2 = st.columns(2)
        
        with col1:
            # 2. Battery Capacity
            battery_capacity = st.number_input(
                "Battery Capacity (kWh)",
                min_value=0.0,
                max_value=100.0,
                value=50.0,
                help="Enter the battery capacity in kilowatt-hours"
            )
            
            # 3. Energy Consumed
            energy_consumed = st.number_input(
                "Energy Consumed (kWh)",
                min_value=0.0,
                max_value=100.0,
                value=30.0,
                help="Enter the amount of energy consumed in kilowatt-hours"
            )
            
            # 4. Charging Duration
            charging_duration = st.number_input(
                "Charging Duration (hours)",
                min_value=0.0,
                max_value=24.0,
                value=2.0,
                help="Enter the charging duration in hours"
            )
            
            # 5. Charging Rate
            charging_rate = st.number_input(
                "Charging Rate (kW)",
                min_value=0.0,
                max_value=350.0,
                value=50.0,
                help="Enter the charging rate in kilowatts"
            )
            
            # 6. Charging Cost
            charging_cost = st.number_input(
                "Charging Cost (USD)",
                min_value=0.0,
                max_value=100.0,
                value=10.0,
                help="Enter the charging cost in USD"
            )
        
        with col2:
            # 7. Time of Day
            time_of_day = st.selectbox(
                "Time of Day",
                options=[0, 1, 2, 3],
                format_func=lambda x: {
                    0: "Afternoon",
                    1: "Evening",
                    2: "Morning",
                    3: "Night"
                }[x],
                help="Select the time of day when charging occurred"
            )
            
            # 8. Day of Week
            day_of_week = st.selectbox(
                "Day of Week",
                options=[0, 1, 2, 3, 4, 5, 6],
                format_func=lambda x: {
                    0: "Friday",
                    1: "Monday",
                    2: "Saturday",
                    3: "Sunday",
                    4: "Thursday",
                    5: "Tuesday",
                    6: "Wednesday"
                }[x],
                help="Select the day of the week"
            )
            
            # 9. State of Charge (Start)
            soc_start = st.number_input(
                "State of Charge (Start %)",
                min_value=0,
                max_value=100,
                value=20,
                help="Select the initial state of charge"
            )
            
            # 10. State of Charge (End)
            soc_end = st.number_input(
                "State of Charge (End %)",
                min_value=0,
                max_value=100,
                value=80,
                help="Select the final state of charge"
            )
            
            # 11. Charger Type
            charger_type = st.selectbox(
                "Charger Type",
                options=[0, 1, 2],
                format_func=lambda x: {
                    0: "DC Fast Charger",
                    1: "Level-1",
                    2: "Level-2"
                }[x],
                help="Select the type of charger used"
            )
            
            # 12. User Type
            user_type = st.selectbox(
                "User Type",
                options=[0, 1, 2],
                format_func=lambda x: {
                    0: "Casual Driver",
                    1: "Commuter",
                    2: "Long Distance Traveler"
                }[x],
                help="Select the type of user"
            )
        
        # Submit button
        submit_button = st.form_submit_button("Predict Charging Cost")
        
        # If form is submitted
        if submit_button:
            # Here you would typically make a prediction using your model
            input_data=(["vehicle_model","battery_capacity","energy_consumed","charging_duration","charging_rate","charging_cost","time_of_day","day_of_week","soc_start","soc_end","charger_type","user_type"])
            input_data_as_numpy_array=np.asarray(input_data)
            input_data_reshaped=input_data_as_numpy_array.reshape(1,-1)
            prediction_output=model.predict(input_data_reshaped)
            
            # For demonstration, we'll just show the input values
            st.success("Prediction Complete!")
            
            st.subheader("Input Summary:")
            col1, col2 = st.columns(2)
            
            with col1:
                st.write(f"Vehicle Model: {vehicle_model}")
                st.write(f"Battery Capacity: {battery_capacity} kWh")
                st.write(f"Energy Consumed: {energy_consumed} kWh")
                st.write(f"Charging Duration: {charging_duration} hours")
                st.write(f"Charging Rate: {charging_rate} kW")
                st.write(f"Charging Cost: ${charging_cost}")
            
            with col2:
                st.write(f"Time of Day: {time_of_day}")
                st.write(f"Day of Week: {day_of_week}")
                st.write(f"Initial SoC: {soc_start}%")
                st.write(f"Final SoC: {soc_end}%")
                st.write(f"Charger Type: {charger_type}")
                st.write(f"User Type: {user_type}")
            
            # Display predicted cost (placeholder)
            st.subheader("Predicted Charging Cost:")
            st.metric(label="Estimated Cost", value=f"${prediction_output:.2f}")

if __name__ == "__main__":
    main()

2024-11-24 13:50:08.562 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:50:08.563 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:50:08.564 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:50:08.565 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:50:08.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:50:08.567 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:50:08.568 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:50:08.568 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [208]:
input_data=(2,100.000000,12.339275,3.133652,30.677735,21.128448,2,1,10.115778,84.664344,1,0)
input_data_as_numpy_array=np.asarray(input_data)
input_data_reshaped=input_data_as_numpy_array.reshape(1,-1)
prediction=model.predict(input_data_reshaped)
prediction

c:\Users\jk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([21.128448])